In [13]:
from dotenv import load_dotenv
import os

# ✅ Load .env file
loaded = load_dotenv()

# ✅ Check if loading was successful
print("🔄 .env Loaded:", loaded)

# ✅ Retrieve the API key
api_key = os.getenv("OPENAI_KEY")

if api_key:
    print("✅ OpenAI API Key loaded successfully!")
else:
    print("❌ Error: API Key not found. Check your .env file.")


🔄 .env Loaded: True
✅ OpenAI API Key loaded successfully!


In [2]:
import dspy
from datasets import load_dataset
import os
from dotenv import load_dotenv

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

True

In [4]:
# 1. Dataset Preparation
def prepare_imdb_data(split, num_samples=200):
    dataset = load_dataset("imdb", split=split)
    return [
        dspy.Example(
            text=example["text"],
            sentiment="positive" if example["label"] else "negative"
        ).with_inputs("text")
        for example in dataset.shuffle(seed=42).select(range(num_samples))
    ]

In [5]:
# 2. Problem Definition
class SentimentClassification(dspy.Signature):
    """Classify movie reviews as positive or negative."""
    text = dspy.InputField(desc="movie review text")
    sentiment = dspy.OutputField(
        desc="sentiment label (positive or negative)",
        choices=["positive", "negative"]
    )

In [6]:
# 3. Evaluation Metric with Error Handling
def sentiment_metric(example, pred, trace=None):
    """Robust metric with type checking"""
    try:
        return int(pred.sentiment.lower() == example.sentiment.lower())
    except AttributeError:
        return 0

In [7]:
# 4. Baseline Model
class BaselineClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(SentimentClassification)
    
    def forward(self, text):
        return self.predict(text=text)

In [8]:
# 5. Optimized Model
class OptimizedClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.ChainOfThought(SentimentClassification)
    
    def forward(self, text):
        return self.predict(text=text)

In [9]:
train_data = prepare_imdb_data("train", 150)
test_data = prepare_imdb_data("test", 50)


In [10]:
api_key = os.getenv("OPENAI_KEY")
gpt = dspy.LM("openai/gpt-3.5-turbo", max_tokens=256, api_key=api_key)
dspy.configure(lm=gpt)

In [11]:
baseline = BaselineClassifier()
evaluator = dspy.evaluate.Evaluate(
    devset=test_data,
    metric=sentiment_metric,
    num_threads=2
)
baseline_acc = evaluator(baseline)

2025/03/11 11:24:24 INFO dspy.evaluate.evaluate: Average Metric: 49 / 50 (98.0%)


In [12]:
# Optimization with MIPROv2
optimizer = dspy.MIPROv2(
    metric=sentiment_metric,
    max_bootstrapped_demos=2
)

optimized = optimizer.compile(
    OptimizedClassifier(),
    trainset=train_data[:100]  # Use subset for stability
)

# Evaluate optimized model
optimized_acc = evaluator(optimized)

# Results
print(f"\n{' Results ':=^40}")
print(f"| {'Baseline Accuracy':<20} | {baseline_acc:.2%} |")
print(f"| {'Optimized Accuracy':<20} | {optimized_acc:.2%} |")

2025/03/11 11:25:03 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/11 11:25:03 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/11 11:25:03 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=10 sets of demonstrations...


Bootstrapping set 1/10
Bootstrapping set 2/10
Bootstrapping set 3/10


 10%|█         | 2/20 [00:02<00:19,  1.08s/it]


Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 4/10


 10%|█         | 2/20 [00:01<00:15,  1.19it/s]


...

2025/03/11 12:24:31 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 8'].
2025/03/11 12:24:31 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [92.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 92.0, 0.0, 92.0, 0.0, 0.0, 96.0, 96.0, 96.0, 96.0, 96.0, 0.0, 88.0, 92.0, 92.0, 92.0, 96.0, 92.0, 64.0, 96.0, 92.0, 96.0, 0.0]
2025/03/11 12:24:31 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [93.75, 0.0, 0.0, 90.0]
2025/03/11 12:24:31 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 93.75
2025/03/11 12:24:31 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/03/11 12:24:31 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 34 / 34 - Full Evaluation =====
2025/03/11 12:24:31 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 96.0) from minibatch trials...


Exception occurred: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-nS9hIqZFvGLn1kCX8ZWaZkba on tokens per min (TPM): Limit 200000, Used 196570, Requested 7017. Please try again in 1.076s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Average Metric: 75.00 / 80 (93.8%): 100%|██████████| 80/80 [01:41<00:00,  1.26s/it]

2025/03/11 12:26:12 INFO dspy.evaluate.evaluate: Average Metric: 75 / 80 (93.8%)
2025/03/11 12:26:12 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [93.75, 0.0, 0.0, 90.0, 93.75]
2025/03/11 12:26:12 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 93.75
2025/03/11 12:26:12 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/03/11 12:26:12 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/03/11 12:26:12 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 93.75!


2025/03/11 12:26:40 INFO dspy.evaluate.evaluate: Average Metric: 50 / 50 (100.0%)



=============== Results ================
| Baseline Accuracy    | 9800.00% |
| Optimized Accuracy   | 10000.00% |
